### Q1
1. Three Householder transformation are required due to it's a $4\times 3$ matrix.
2. $[2.0 0 0 0]^T$.
3. The 1st column of $A$ will remain unchanged.
4. $(4-1)+(3-1)+(2-1)+(1-1)=6$ times rotations are needed.

### Q2

In [2]:
using LinearAlgebra,Test, Random

struct GivensMatrix{T} <: AbstractArray{T, 2}
	c::T
	s::T
	i::Int
	j::Int
	n::Int
end

Base.size(g::GivensMatrix) = (g.n, g.n)

Base.size(g::GivensMatrix, i::Int) = i == 1 || i == 2 ? g.n : 1

function givens(A, i, j)
	x, y = A[i, 1], A[j, 1]
	norm = sqrt(x^2 + y^2)
	c = x/norm
	s = y/norm
	return GivensMatrix(c, s, i, j, size(A, 1))
end

function left_mul!(A::AbstractMatrix, givens::GivensMatrix)
	@inbounds for col in 1:min(size(A, 2),3)
		vi, vj = A[givens.i, col], A[givens.j, col]
		A[givens.i, col] = vi * givens.c + vj * givens.s
		A[givens.j, col] = -vi * givens.s + vj * givens.c
	end
	return A
end
function right_mul!(A::AbstractMatrix, givens::GivensMatrix)
	@inbounds for row in 1:size(A, 1)
		vi, vj = A[row, givens.i], A[row, givens.j]
		A[row, givens.i] = vi * givens.c + vj * givens.s
		A[row, givens.j] = -vi * givens.s + vj * givens.c
	end
	return A
end

function givens_trisym!(Q::AbstractMatrix,A::AbstractMatrix)
    m, n = size(A)
	if m == 1
		return Q, A
	else
		g = givens(A, 1, 2)
		left_mul!(A, g)
		right_mul!(Q, g)

		givens_trisym!(view(Q, :, 2:m), view(A, 2:m, 2:n))
		return Q, A
	end
end

@testset "test givens" begin
    n = 5
    triSym = diagm(randn(Float64,n))
    off_diag = randn(Float64,n-1,n-1)
    for i in 1:n-1
        triSym[i+1,i] = off_diag[i]
        triSym[i,i+1] = off_diag[i]
    end

    display(triSym)
    triSym_me= copy(triSym)
    my_Q,my_R = givens_trisym!(Matrix{Float64}(I,n,n),triSym_me)
    display(my_Q)
    display(my_R)
    @test my_Q * my_R  ≈ triSym
    @test my_Q * my_Q' ≈ I
end

5×5 Matrix{Float64}:
 -0.500336    0.0463096   0.0        0.0          0.0
  0.0463096  -1.13623     0.256178   0.0          0.0
  0.0         0.256178   -0.417988   0.882033     0.0
  0.0         0.0         0.882033  -2.10672     -0.00696791
  0.0         0.0         0.0       -0.00696791   0.534923

5×5 Matrix{Float64}:
 -0.995744  -0.089871  -0.00755351   0.0179287  -0.00622382
  0.092163  -0.97098   -0.0816093    0.193704   -0.0672431
  0.0        0.221631  -0.360599     0.855901   -0.297121
  0.0        0.0        0.929113     0.349345   -0.121273
  0.0        0.0        0.0         -0.327945   -0.944697

5×5 Matrix{Float64}:
 0.502475  -0.150831      0.0236101   0.0           0.0
 0.0        1.15588      -0.341383    0.195486      0.0
 0.0        2.77556e-17   0.949328   -2.27544      -0.00647398
 0.0        0.0           0.0         0.0212472    -0.17786
 0.0        0.0           0.0         8.67362e-19  -0.504495

Test Summary: | Pass  Total  Time
test givens   |    2      2  1.3s


Test.DefaultTestSet("test givens", Any[], 2, false, false, true, 1.683561520903971e9, 1.683561522183982e9)

#### Credits to Yusheng Zhao